In [43]:
import duckdb
import pandas as pd
import os

In [44]:
df_dist_urg = duckdb.read_csv("./data/raw/dist_urgence.csv",skiprows=2)
df_dist_urg

┌─────────┬─────────────────────────┬─────────────────────────────────────────────┐
│  Code   │         Libellé         │ Distance à la structure la plus proche 2024 │
│ varchar │         varchar         │                   varchar                   │
├─────────┼─────────────────────────┼─────────────────────────────────────────────┤
│ 01001   │ L'Abergement-Clémenciat │ 26                                          │
│ 01002   │ L'Abergement-de-Varey   │ 15                                          │
│ 01004   │ Ambérieu-en-Bugey       │ 0                                           │
│ 01005   │ Ambérieux-en-Dombes     │ 18                                          │
│ 01006   │ Ambléon                 │ 11                                          │
│ 01007   │ Ambronay                │ 7                                           │
│ 01008   │ Ambutrix                │ 5                                           │
│ 01009   │ Andert-et-Condon        │ 7                                     

In [45]:
mapping = {
    "Code": "code_insee",
    "Libellé": "nom_commune",
    "Distance à la structure la plus proche 2024": "dist_urgence_min"
}

df_dist_urg = df_dist_urg.df()  # conversion DuckDB → pandas
df_dist_urg = df_dist_urg.rename(columns=mapping)



In [46]:
#on retraduis en objet duckdb
df_dist_urg = duckdb.from_df(df_dist_urg)

In [47]:
df_com = duckdb.read_csv("./data/processed/communes_france_2025.csv",sep=",")
df_com

┌────────────┬─────────────────────────┬─────────────────────────┬──────────┬──────────────────────┬──────────┬──────────────┬───────────┬───────────────────────────────────────────┬────────────┬────────────────┬─────────────────┐
│ code_insee │      nom_standard       │     nom_sans_accent     │ reg_code │       reg_nom        │ dep_code │   dep_nom    │ epci_code │                 epci_nom                  │ population │ superficie_km2 │ densite_hab_km2 │
│  varchar   │         varchar         │         varchar         │  int64   │       varchar        │ varchar  │   varchar    │  varchar  │                  varchar                  │   int64    │     int64      │     double      │
├────────────┼─────────────────────────┼─────────────────────────┼──────────┼──────────────────────┼──────────┼──────────────┼───────────┼───────────────────────────────────────────┼────────────┼────────────────┼─────────────────┤
│ 01001      │ L'Abergement-Clémenciat │ l-abergement-clemenciat │       84 

In [48]:
query = """
SELECT 
    DISTINCT epci_code,
    ROUND(AVG(TRY_CAST(dist_urgence_min AS DOUBLE)),2) AS dist_urgence_moyenne
FROM df_com
LEFT JOIN df_dist_urg
ON df_com.code_insee = df_dist_urg.code_insee
GROUP BY epci_code
"""

df_dist_soin = duckdb.sql(query)

In [49]:
df_dist_pharma = duckdb.read_csv("./data/raw/dist_pharma.csv",skiprows=2)
df_dist_pharma

┌─────────┬─────────────────────────┬─────────────────────────────────────────────┐
│  Code   │         Libellé         │ Distance à la pharmacie la plus proche 2024 │
│ varchar │         varchar         │                   varchar                   │
├─────────┼─────────────────────────┼─────────────────────────────────────────────┤
│ 01001   │ L'Abergement-Clémenciat │ 5                                           │
│ 01002   │ L'Abergement-de-Varey   │ 15                                          │
│ 01004   │ Ambérieu-en-Bugey       │ 0                                           │
│ 01005   │ Ambérieux-en-Dombes     │ 11                                          │
│ 01006   │ Ambléon                 │ 11                                          │
│ 01007   │ Ambronay                │ 7                                           │
│ 01008   │ Ambutrix                │ 5                                           │
│ 01009   │ Andert-et-Condon        │ 7                                     

In [50]:
mapping = {
    "Code": "code_insee",
    "Libellé": "nom_commune",
    "Distance à la pharmacie la plus proche 2024": "dist_pharma_min"
}

df_dist_pharma = df_dist_pharma.df()  # conversion DuckDB → pandas
df_dist_pharma = df_dist_pharma.rename(columns=mapping)

df_dist_pharma = duckdb.from_df(df_dist_pharma)

In [51]:
query ="""
SELECT
    DISTINCT epci_code,
    ROUND(AVG(TRY_CAST(dist_pharma_min AS DOUBLE)),2) AS dist_pharma_moyenne
FROM df_com
LEFT JOIN df_dist_pharma
ON df_com.code_insee = df_dist_pharma.code_insee
GROUP BY epci_code
"""

df_dist_pharma_moy = duckdb.sql(query)

In [52]:
query = """ 
SELECT
    d.epci_code as siren,
    d.dist_urgence_moyenne,
    p.dist_pharma_moyenne
FROM df_dist_soin d
LEFT JOIN df_dist_pharma_moy p
ON d.epci_code = p.epci_code
"""

df_dist_soin_final = duckdb.sql(query)
df_dist_soin_final

┌───────────┬──────────────────────┬─────────────────────┐
│   siren   │ dist_urgence_moyenne │ dist_pharma_moyenne │
│  varchar  │        double        │       double        │
├───────────┼──────────────────────┼─────────────────────┤
│ 200071751 │                17.01 │               12.64 │
│ 240200501 │                19.34 │               12.16 │
│ 200068765 │                28.82 │               18.85 │
│ 200041465 │                46.24 │               16.34 │
│ 200040491 │                12.71 │                7.47 │
│ 240700815 │                18.38 │                9.77 │
│ 241000405 │                43.56 │                8.96 │
│ 200071777 │                35.67 │               29.31 │
│ 241200542 │                37.09 │               33.27 │
│ 200065589 │                19.83 │                7.83 │
│     ·     │                   ·  │                  ·  │
│     ·     │                   ·  │                  ·  │
│     ·     │                   ·  │                  · 

In [53]:
df_dist_soin_final.write_csv("./data/processed/distance_lieu_soin_epci.csv")